In [ ]:
import timm
import torch
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import confusion_matrix
from torchvision import transforms, datasets


In [ ]:
print(timm.__version__)
print(torch.__version__)
print(torchvision.__version__)

In [ ]:
def create_vit(model_name):
    if model_name == 'tiny': model = timm.create_model('vit_tiny_patch16_224', pretrained=False, num_classes=4)
    if model_name == 'small': model = timm.create_model('vit_small_patch16_224', pretrained=False, num_classes=4)
    if model_name == 'base': model = timm.create_model('vit_base_patch16_224', pretrained=False, num_classes=4)

    return model

In [ ]:
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

dataset = datasets.ImageFolder(root='/Users/aravdhoot/Parkinson-Project/non-keyframes/energy_images', transform=transform)

total_size = len(dataset)
train_size = int(total_size * 0.8) 
validation_size = int(total_size * 0.1) 
test_size = total_size - train_size - validation_size
generator = torch.Generator().manual_seed(0) 
train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size], generator=generator)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
model = create_vit('tiny')

# Model summary to check architecture
print(model)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'mps')
print(f"Using device: {device}")

# Transfer the model to the GPU
model.to(device)

num_epochs = 150  # Set the number of epochs
batch_size = 64
chs = 200 * batch_size  # Similar to the previous code

early_stopping = False  # Set to True for early stopping
patience = 20
best_val_accuracy = 0.0
counter = 0

train_loss_list = []
val_loss_list = []
train_accuracy_list = []
val_accuracy_list = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")

    # Training step
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    counter = 0
    for batch_idx, (images, labels) in enumerate(train_loader):
        print(counter)
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        counter += 1 

        if (batch_idx + 1) % chs == 0 or (batch_idx + 1) == len(train_loader):
            train_loss = running_loss / (batch_idx + 1)
            train_accuracy = 100 * correct / total
            train_loss_list.append(train_loss)
            train_accuracy_list.append(train_accuracy)

            print(f"Training Batch {batch_idx+1}, Loss: {train_loss:.6f}, Accuracy: {train_accuracy:.2f}")

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(val_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            if early_stopping and (batch_idx + 1) % chs == 0:
                val_accuracy = 100 * val_correct / val_total
                val_loss /= (batch_idx + 1)
                val_accuracy_list.append(val_accuracy)
                val_loss_list.append(val_loss)

                print(f"Validation Batch {batch_idx+1}, Loss: {val_loss:.6f}, Accuracy: {val_accuracy:.2f}")

                # Early stopping check
                if val_accuracy > best_val_accuracy:
                    best_val_accuracy = val_accuracy
                    counter = 0
                else:
                    counter += 1
                    if counter >= patience:
                        print(f"Early stopping at epoch {epoch+1} due to no improvement in validation accuracy.")
                        break

    if early_stopping and counter >= patience:
        break

print("Training complete.")

In [ ]:
model.eval()

test_loss = 0.0
test_correct = 0
test_total = 0
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch_idx, (images, labels) in enumerate(test_loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        print(f"Test Batch {batch_idx+1}, Loss: {loss.item():.6f}, Accuracy: {100 * test_correct / test_total:.2f}%")

test_loss /= len(test_loader)
test_accuracy = 100 * test_correct / test_total

print(f"Test Set - Loss: {test_loss:.6f}, Accuracy: {test_accuracy:.2f}%")

cm = confusion_matrix(all_labels, all_predictions)
print("Confusion Matrix:")
print(cm)